In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from PIL import Image
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#region_of_interest function will return only image that contains fully lane lines
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    mask_color = 255
    cv2.fillPoly(mask, vertices, mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
    line_img = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
    img = np.copy(img)
    
    if lines is None:
        return

    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
            
# Merge the image with the lines onto the original
    img = cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)

    return img

def pipeline(image):
    
#getting an image information (resolution)
    height = image.shape[0]
    width = image.shape[1]

#Defining the region of interest
    roiVertices = [(0, height),(width / 2, height / 2),(width, height)]

#rgb to grayScale conversion 
    grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

#Canny Edge Detection
    cannyEdge= cv2.Canny(grayScale, 100, 200)
   
#Cropping of image, after applying canny edge transform  
    cropped_image = region_of_interest(cannyEdge,np.array([roiVertices],np.int32))
    
#Hough Transform function helps to extract feature of line shapes from Cropped_image
    laneLines = cv2.HoughLinesP(cropped_image,rho=6,theta=np.pi / 60,threshold=100,lines=np.array([]),minLineLength=40, maxLineGap=20)

#Grouping of left and right lane markings, that helps to construct further pipeline in image 
    left_x = []
    left_y = []
    right_x = []
    right_y = []
 
    for line in laneLines:
        for x1, y1, x2, y2 in line:
#Calculating a Slope
            slope = (y2 - y1) / (x2 - x1)
            if math.fabs(slope) < 0.5: # absolute slope (is not consider)
                continue
#Slope is negative, left group
            if slope <= 0:
                left_x.extend([x1, x2])
                left_y.extend([y1, y2])
#Slope is positive, right group
            else:
                right_x.extend([x1, x2])
                right_y.extend([y1, y2])
                
'''Average the lines in each group, below function fit in location of image with proper orientation
(line start from bottom image to horizon)'''

    min_y = int(image.shape[0] * (3 / 5))
    max_y = int(image.shape[0])
    
'''The polyfit and poly1d operations can generate
 a linear function that match the two given spaces (x and y) for each group'''

    left = np.poly1d(np.polyfit(left_y,left_x,deg=1))
 
    leftStart = int(left(max_y))
    leftEnd = int(left(min_y))
 
    right = np.poly1d(np.polyfit(right_y,right_x,deg=1))
 
    rightStart = int(right(max_y))
    rightEnd = int(right(min_y))
    
# function call to draw_lines
    out = draw_lines(image,
        [[
            [leftStart, max_y, leftEnd, min_y],
            [rightStart, max_y, rightEnd, min_y],
        ]],
        thickness=5)

    return out

## draw lanes over a video

white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")

# clip1 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!

## Write a video output 
white_clip.write_videofile(white_output, audio=False)

## play inline output video 
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))
# image = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')
# out = pipeline(image)
# plt.imshow(out)
# plt.savefig('test_images_output/whiteCarLaneSwitch.jpg')
